## Mutational burden analysis for denv2 intra-host genetic variants data

In [1]:
import os

path = ""
file = "data.csv"
csv = os.path.join(path, file)

In [2]:
import pandas as pd
csv_data = pd.read_csv(csv).fillna(0)
csv_data.head()

,gene,pos,ref,alt,aa_subs,160,161,162,163,166,...,200,201,202,203,204,137,139,140,143,149
0,5UTR,67,A,T,-,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
1,5UTR,70,G,GT,-,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
2,5UTR,95,T,A,-,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,5UTR,96,G,A,-,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,C,102,C,T,-,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
# samples by clinical classification
DF = list(map(str, [  # dengue fever
    160,
    161,
    162,
    163,
    166,
    167,
    168,
    169,
    170,
    171,
    172,
    173,
    174,
    175,
    177,
    178,
    179,
    180,
    181,
    182,
    183,
    184,
    141,
    142,
    145,
    146,
    151,
    154,
    155,
    158,
    159,
]))


WS = list(map(str, [  # warning signs
    185,
    186,
    187,
    188,
    189,
    190,
    191,
    192,
    193,
    207,
    205,
    206,
    138,
    144,
    147,
    148,
    153,
    156,
    157,
]))


SD = list(map(str, [  # severe dengue
    208,
    209,
    194,
    195,
    196,
    197,
    198,
    199,
    200,
    201,
    202,
    203,
    204,
    137,
    139,
    140,
    143,
    149,
]))

In [4]:
annots = {
    "5UTR": (1, 96),
    "C": (97, 438),
    "prM/M": (439, 936),
    "E": (937, 2421),
    "NS1": (2422, 3477),
    "NS2A": (3478, 4131),
    "NS2B": (4132, 4521),
    "NS3":  (4522, 6375),
    "NS4A": (6376, 6825),
    "NS4B": (6826, 7569),
    "NS5": (7570, 10269),
    "3UTR": (10273, 10723),
}

In [5]:
# separate by class
df_data = csv_data[csv_data.columns[:5]].assign(df = csv_data[DF].sum(axis=1))
ws_data = csv_data[csv_data.columns[:5]].assign(ws = csv_data[WS].sum(axis=1))
sd_data = csv_data[csv_data.columns[:5]].assign(sd = csv_data[SD].sum(axis=1))

# remove lines with no mutations
df_data = df_data[df_data.df != 0.0]
ws_data = ws_data[ws_data.ws != 0.0]
sd_data = sd_data[sd_data.sd != 0.0]

In [6]:
import numpy as np

def get_mutations(region_len, df):
    """Get unique mutations count in each region."""
    x = []  # mutation count in each region
    n = []  # number of bases in each region
    k = annots["3UTR"][1] - annots["5UTR"][0]
    labels_str = []
    labels_int = []
    
    for region in range(0, k, region_len):
        below = df[df["pos"] < region + region_len]
        above = below[below["pos"] >= region]
        x.append(above.shape[0])
        
        if region + region_len > k:
            n.append(k - region)
#             labels_str.append("({0:05d}, {1:05d})".format(region, region + (k - region)))
            labels_str.append("({}, {})".format(region + 1, region + (k - region) + 1))
            labels_int.append((region + 1, region + (k - region) + 1))
        else:
            n.append(region_len)
#             labels_str.append("({0:05d}, {1:05d})".format(region, region + region_len))
            labels_str.append("({}, {})".format(region + 1, region + region_len + 1))
            labels_int.append((region + 1, region + region_len + 1))
    
    return x, n, labels_str, labels_int

In [7]:
# define window size
REGION_LEN = 3 * 6

In [8]:
muts_df, windows_df, labels_str_df, labels_int_df = get_mutations(REGION_LEN, df_data)
muts_ws, windows_ws, labels_str_ws, labels_int_ws = get_mutations(REGION_LEN, ws_data)
muts_sd, windows_sd, labels_str_sd, labels_int_sd = get_mutations(REGION_LEN, sd_data)

# get density (divide by region len) 
df_dens = np.asarray(muts_df) / np.asarray(windows_df)
ws_dens = np.asarray(muts_ws) / np.asarray(windows_ws)
sd_dens = np.asarray(muts_sd) / np.asarray(windows_sd)

## Plot heatmaps

In [9]:
import matplotlib.pyplot as plt
%matplotlib inline
%matplotlib widget

def plot_heatmap(df_dens, ws_dens, sd_dens, annots, labels_str, labels_int, gene):
    inds = ((np.array(labels_int)[:, 0] >= annots[gene][0]).astype(int)  * (np.array(labels_int)[:, 1] <= annots[gene][1] + REGION_LEN).astype(int)).astype(bool)
    heat = np.array([df_dens[inds], ws_dens[inds], sd_dens[inds]])
    
    fig, ax = plt.subplots()
    fig.set_size_inches(25.5, 5.5)
    
    tot = np.array([df_dens, ws_dens, sd_dens])
    im = ax.imshow(heat, interpolation='nearest', aspect='auto', cmap='YlOrBr', vmin=np.min(tot), vmax=np.max(tot))
    
    ax.set_yticks(range(3))
    ax.set_yticklabels(['DF', 'WS', 'SD'])
    ax.set_xticks(range(len(np.array(labels_str)[inds])))
    ax.set_xticklabels(np.array(labels_str)[inds])
    ax.set_title(f"{gene}")
    plt.xticks(rotation=90)
    plt.tight_layout()
    plt.colorbar(im)

In [10]:
# plot heatmaps
for gene in annots.keys():
    plot_heatmap(df_dens, ws_dens, sd_dens, annots, labels_str_df, labels_int_df, gene=gene)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

...

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Hotspot candidates with the binomial model

In [11]:
def get_bg_mutation_rates_per_gene(annots, df):
    """Calculate background mutation rates per gene."""
    rates = {
        "5UTR": 0,
        "C": 0,
        "prM/M": 0,
        "E": 0,
        "NS1": 0,
        "NS2A": 0,
        "NS2B": 0,
        "NS3":  0,
        "NS4A": 0,
        "NS4B": 0,
        "NS5": 0,
        "3UTR": 0,
    }
    
    for key in rates.keys():
        abv = df[df['pos'] >= annots[key][0]]
        blw = abv[abv['pos'] <= annots[key][1]]
        rates[key] = blw.shape[0] / (annots[key][1] - annots[key][0] + 1)

    return rates

In [12]:
def get_bg_mutation_rate(x, n):
    """Global estimation of the mutation rate.
    
    x : List[int]
        mutation count in each region
    n : List[int]
        number of bases in each region
        
    """
    x = np.asarray(x)
    n = np.asarray(n)
    assert x.shape[0] == n.shape[0], "# of regions must match"
    
    return x.sum()/n.sum()

In [13]:
# get background mutation rates per gene
bg_df = get_bg_mutation_rates_per_gene(annots, df_data)
bg_ws = get_bg_mutation_rates_per_gene(annots, ws_data)
bg_sd = get_bg_mutation_rates_per_gene(annots, sd_data)

# global bg rates
p_df = get_bg_mutation_rate(muts_df, windows_df)
p_ws = get_bg_mutation_rate(muts_ws, windows_ws)
p_sd = get_bg_mutation_rate(muts_sd, windows_sd)

In [14]:
from scipy import stats
import re

def binom_test(x, n, p, bg_genes, labels_int, annots):
    """Perform a binomial test for each region."""
    assert len(x) == len(n) == len(labels_int)
    
    p_vals = []
    for r in range(len(x)):
        flag = False
        for key in annots.keys():
            if labels_int[r][0] >= annots[key][0] and labels_int[r][1] <= annots[key][1]:        
                p_val = stats.binom_test(x=x[r], n=n[r], p=bg_genes[key], alternative="greater")
                p_vals.append(p_val)
                flag = True
        if flag is False:
            # its a region in between two genes
            # use 'default' mutation rate
            p_val = stats.binom_test(x=x[r], n=n[r], p=p, alternative="greater")
            p_vals.append(p_val)
            
    return p_vals

In [15]:
import statsmodels.stats.multitest as stm
from prettytable import PrettyTable

def print_table_of_significant_regions(muts, windows, labels, class_name, annots, p, bg_genes):
    p_vals = stm.multipletests(np.asarray(binom_test(muts, windows, p, bg_genes, labels, annots)), method='fdr_bh')[1]
    
    # to np array
    muts = np.asarray(muts)
    windows = np.asarray(windows)
    labels = np.asarray(labels)
    
    p_inds = np.arange(p_vals.shape[0])[p_vals < 0.05]  # select regions

    flags = {
        "5UTR": False,
        "C": False,
        "prM/M": False,
        "E": False,
        "NS1": False,
        "NS2A": False,
        "NS2B": False,
        "NS3":  False,
        "NS4A": False,
        "NS4B": False,
        "NS5": False,
        "3UTR": False,
    }

    tab = PrettyTable()
    tab.title = f"{class_name} class"
    tab.field_names = ["region", "length", "mutations", "adj. p-value", "index"]
    tab.vrules = 0
    tab.align = "l"
    for i in p_inds:
        flag = False
        for r in annots.keys():
            if labels[i][0] > annots[r][0] and labels[i][1] < annots[r][1]:
                flag = True
                if not flags[r]:
                    tab.add_row(["----", "----", "<" + r + ">", "----", "----"])
                    flags[r] = True
        tab.add_row([f"{labels[i][0]}-{labels[i][0]}", windows[i], muts[i], round(p_vals[i], 5), i])
    print(tab)
    print()

In [17]:
# print tables
print_table_of_significant_regions(muts_df, windows_df, labels_int_df, "DF", annots, p_df, bg_df)
print_table_of_significant_regions(muts_ws, windows_ws, labels_int_ws, "WS", annots, p_ws, bg_ws)
print_table_of_significant_regions(muts_sd, windows_sd, labels_int_sd, "SD", annots, p_sd, bg_sd)

+---------------------------------------------------------+
|                         DF class                        |
+---------------------------------------------------------+
| region        length   mutations   adj. p-value   index |
+---------------------------------------------------------+
| ----          ----     <NS1>       ----           ----  |
| 2485-2485     18       9           0.04716        138   |
| ----          ----     <NS3>       ----           ----  |
| 5833-5833     18       10          0.04716        324   |
| ----          ----     <3UTR>      ----           ----  |
| 10603-10603   18       8           0.04716        589   |
+---------------------------------------------------------+

+----------------------------------------------------+
|                      WS class                      |
+----------------------------------------------------+
| region   length   mutations   adj. p-value   index |
+----------------------------------------------------+
+---